In [1]:
import pandas as pd
from datetime import datetime

In [2]:
df = pd.read_csv('texasma.csv')
df.head(2)

,Crash ID,Average Daily Traffic Amount,Average Daily Traffic Year,City,County,Crash Date,Crash Death Count,Crash Month,Crash Severity,Crash Time,...,Person Blood Alcohol Content Test Result,Person Death Count,Person Drug Test Result,Person Ethnicity,Person Gender,Person Helmet,Person Injury Severity,Person Not Injured Count,Person Total Injury Count,Person Type
0,11825679,21549,2018,SAN ANTONIO,BEXAR,1/1/11,0,1,B - SUSPECTED MINOR INJURY,2230,...,No Data,0,97 - NOT APPLICABLE,H - HISPANIC,1 - MALE,1 - NOT WORN,B - SUSPECTED MINOR INJURY,0,1,5 - DRIVER OF MOTORCYCLE TYPE VEHICLE
1,11825679,21549,2018,SAN ANTONIO,BEXAR,1/1/11,0,1,B - SUSPECTED MINOR INJURY,2230,...,No Data,0,No Data,H - HISPANIC,2 - FEMALE,1 - NOT WORN,C - POSSIBLE INJURY,0,1,6 - PASSENGER/OCCUPANT ON MOTORCYCLE TYPE VEHICLE


In [3]:
df = df.drop(columns=['Crash ID', 'Average Daily Traffic Amount', 'Average Daily Traffic Year', 'Highway Number',
                     'Surface Condition', 'Surface Type', 'Vehicle Hit and Run Flag',
                     'Person Blood Alcohol Content Test Result', 'Person Drug Test Result',
                     'Crash Month', 'Crash Year', 'Number of Lanes', 'License Plate State',
                     'Driver License Type', 'Crash Severity', 'Unit Death Count', 'Unit Not Injured Count',
                     'Person Death Count', 'Person Injury Severity', 'Person Not Injured Count', 'Person Total Injury Count'])

In [4]:
df = df.rename(str.lower, axis='columns')

In [5]:
df = df.rename(columns = {'crash date':'date', 'day of week':'day',
                         'weather condition':'weather', 'vehicle color':'color', 'vehicle make':'make',
                         'person age':'age', 'person ethnicity':'ethnicity', 'crash death count':'deceased', 'crash time':'time',
                         'crash total injury count':'injured', 'driver license state':'dl_state', 'person gender':'gender',
                         'person helmet':'helmet', 'person type':'driver'})

In [6]:
df = df.replace(to_replace = {'5 - DRIVER OF MOTORCYCLE TYPE VEHICLE', '6 - PASSENGER/OCCUPANT ON MOTORCYCLE TYPE VEHICLE'},
                value = {'driver', 'passenger'})

In [7]:
df = df.replace(to_replace = {'1 - NOT WORN', '99 - UNKNOWN IF WORN'}, value = 'not worn')

In [8]:
df = df.replace(to_replace = {'2 - WORN, DAMAGED', '3 - WORN, NOT DAMAGED', '4 - WORN, UNK DAMAGE'}, value = 'worn')

In [9]:
df = df.replace(to_replace = {'2 - FEMALE'}, value = 'female')

In [10]:
df = df.replace(to_replace = {'1 - MALE'}, value = 'male')

In [11]:
df['gender'] = df['gender'].replace({'99 - UNKNOWN':'male', 'No Data':'male'})

In [12]:
df['ethnicity'] = df['ethnicity'].replace({'No Data':'98 - OTHER'})

In [13]:
df['age'] = df['age'].replace({'No Data':37})

In [14]:
df['make'] = df['make'].replace({'No Data':'unknown', 'UNKNOWN':'unknown'})

In [15]:
df['color'] = df['color'].replace({'No Data':'99 - UNKNOWN'})

In [16]:
df['dl_state'] = df['dl_state'].replace({'No Data':'UN - UNKNOWN'})

In [17]:
df['injured'] = df['injured'].replace({2:1, 3:1, 4:1, 5:1, 6:1, 7:1})

In [18]:
df['deceased'] = df['deceased'].replace({2:0})

In [19]:
df['latitude'] = df['latitude'].replace({'No Data':0})

In [20]:
df['longitude'] = df['longitude'].replace({'No Data':0})

In [21]:
df['weather'] = df['weather'].str[4:]

In [22]:
df['weather'] = df['weather'].str.strip()

In [23]:
df['dl_state'] = df['dl_state'].str[5:]

In [24]:
df['color'] = df['color'].str[6:]

In [25]:
df['ethnicity'] = df['ethnicity'].str[4:]

In [26]:
df['ethnicity'] = df['ethnicity'].str.strip()

In [27]:
df.time = df.time.astype(str)
df['time'] = df['time'].apply(lambda x: x.zfill(4))
df.time = df.time.str[:2] + ':' + df.time.str[-2:]

In [28]:
df['crash_date'] = df['date'] +' '+ df['time']

In [29]:
df['crash_date'] = pd.to_datetime(df['crash_date'])

In [30]:
df = df.set_index('crash_date').sort_index()

In [31]:
convert_dict_int = {'age': int, 'deceased':int, 'injured':int, 'latitude':float, 'longitude':float}
df = df.astype(convert_dict_int)

In [32]:
#get_dummies creates a seperate df of booleans for the identified columns below. Cleaning for the decission tree.
dummy_df = pd.get_dummies(df[['driver', 'helmet', 'gender']], dummy_na=False, drop_first=[True, True])

In [33]:
df = df.drop(columns=['driver', 'helmet', 'gender', 'date', 'time'])

In [34]:
df = pd.concat([df, dummy_df], axis=1)

In [35]:
df = df.rename(columns = {'driver_passenger':'driver', 'helmet_worn':'helmet', 'gender_male':'male'})

In [36]:
convert_dict_int = {'driver': int, 'helmet':int, 'male':int}
df = df.astype(convert_dict_int)

In [37]:
df.head()

,city,county,deceased,injured,day,latitude,longitude,weather,dl_state,color,make,age,ethnicity,driver,helmet,male
crash_date,,,,,,,,,,,,,,,,
2011-01-01 22:30:00,SAN ANTONIO,BEXAR,0,1,SATURDAY,29.358915,-98.566314,CLEAR,TEXAS,BLACK,HONDA,27,HISPANIC,1,0,1
2011-01-01 22:30:00,SAN ANTONIO,BEXAR,0,1,SATURDAY,29.358915,-98.566314,CLEAR,TEXAS,BLACK,HONDA,28,HISPANIC,0,0,0
2011-01-03 12:15:00,SAN ANTONIO,BEXAR,1,0,MONDAY,29.537465,-98.424572,CLOUDY,TEXAS,BLACK,HARLEY-DAVIDSON,53,WHITE,1,0,1
2011-01-05 21:15:00,SAN ANTONIO,BEXAR,0,1,WEDNESDAY,29.513015,-98.535152,CLEAR,TEXAS,BLACK,HARLEY-DAVIDSON,47,WHITE,1,0,1
2011-01-06 12:10:00,AUSTIN,WILLIAMSON,0,1,THURSDAY,30.476422,-97.764810,CLEAR,TEXAS,BLACK,SUZUKI,19,WHITE,1,1,1


In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9055 entries, 2011-01-01 22:30:00 to 2021-10-06 00:40:00
Data columns (total 16 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   city       9055 non-null   object 
 1   county     9055 non-null   object 
 2   deceased   9055 non-null   int64  
 3   injured    9055 non-null   int64  
 4   day        9055 non-null   object 
 5   latitude   9055 non-null   float64
 6   longitude  9055 non-null   float64
 7   weather    9055 non-null   object 
 8   dl_state   9055 non-null   object 
 9   color      9055 non-null   object 
 10  make       9055 non-null   object 
 11  age        9055 non-null   int64  
 12  ethnicity  9055 non-null   object 
 13  driver     9055 non-null   int64  
 14  helmet     9055 non-null   int64  
 15  male       9055 non-null   int64  
dtypes: float64(2), int64(6), object(8)
memory usage: 1.2+ MB


In [41]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
deceased,9055.0,0.038211,0.191716,0.000000,0.000000,0.000000,0.000000,1.000000
injured,9055.0,0.816786,0.386863,0.000000,1.000000,1.000000,1.000000,1.000000
latitude,9055.0,28.355945,8.744257,0.000000,29.496560,30.224826,32.589231,36.500007
longitude,9055.0,-89.458293,27.208685,-106.624732,-98.435634,-97.290132,-95.588053,0.000000
age,9055.0,38.836554,14.643108,0.000000,26.000000,37.000000,50.000000,100.000000
driver,9055.0,0.925235,0.263027,0.000000,1.000000,1.000000,1.000000,1.000000
helmet,9055.0,0.490889,0.499945,0.000000,0.000000,0.000000,1.000000,1.000000
male,9055.0,0.880066,0.324902,0.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
for c in df.columns:
    print ("---- %s ---" % c)
    print (df[c].value_counts())

In [ ]:
df1 = df[['city', 'county', 'latitude', 'longitude']]
df1.head()

In [ ]:
temp_df = df.loc[(df['latitude'] == 'No Data')]
temp_df